In [1406]:
#%matplotlib widget

import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import random
import math



In [1407]:
data_frame = pd.read_csv('../data/cleaned/Stats_1.csv', sep=',', header=None).drop(0)
data_frame = data_frame.drop(data_frame.columns[-1],axis=1)

col_names = ['Unit_Type', 'Unit_Size', 'HP', 'Speed', 'Attack', 'Defense', 'BD', 'AP', 'BMD','MAP', 'AOE', 'Range', 'RT', 'Armor', 'W', 'BL', 'Size', 'APT']
rename_dict = {k: col_names[k] for k in range(18)}
data_frame.rename(columns=rename_dict, inplace=True)

columns_to_convert = data_frame.columns.tolist()
columns_to_convert.remove('W')
columns_to_convert.remove('Unit_Type')


# Convert columns to int
data_frame[columns_to_convert] = data_frame[columns_to_convert].astype(int)


# Changing Speed To D&D
data_frame['Speed'] = round(data_frame['Speed'] / 15 / 1.5) * 1.5
data_frame['Range'] = round(data_frame['Range'] /15 / 1.5) * 1.5
#data_frame['HP'] = data_frame['HP'].astype(int)



# Create Current Hp and Unit_Size columns
data_frame.insert(data_frame.columns.get_loc("Unit_Size"), "Max_Unit_Size", data_frame["Unit_Size"])
data_frame.insert(data_frame.columns.get_loc("HP"), "Max_HP", data_frame["HP"])

# Rename "Unit_Size" column to "Current_Unit_Size"
data_frame.rename(columns={"Unit_Size": "Current_Unit_Size"}, inplace=True)
data_frame.rename(columns={"HP": "Current_HP"}, inplace=True)




In [1408]:
def calculate_chance_to_hit(attacker_type, defender_type, dataframe, Given_Range=[], simulation_precision = 'max'):
    
    if not isinstance(attacker_type, list):
        raise ValueError("Attacker is not a list")
    if not isinstance(attacker_type, list):
        raise ValueError("Defender is not a list")
    if not isinstance(Given_Range, list):
        raise ValueError("Range is not a list")
    
    if Given_Range == []:
        Given_Range == [0] * len(attacker_type)
    
    Chance_To_Hit = []
    Attack_Results = []
    Attack_Damage = []
    Units_Lost = []
    HP_Lost = []
    Index = []
    Unit_That_Died = []

    Non_Attacker = []
    Non_Defender = []
    Non_Range = []

    Simulation_Precision = []
    PIS = 0.4
    k = 0 
    
    for index, item in enumerate(attacker_type):
        try:
            attacker = dataframe.loc[[attacker_type[index]], :]
            defender = dataframe.loc[[defender_type[index]], :]
            given_range = Given_Range[index]

            attacker_attack = attacker['Attack'].values[0]
            defender_defense = defender['Defense'].values[0]

            attacker_range = attacker['Range'].values[0]


            # Checks
            if attacker['Current_Unit_Size'].values[0] <= 0:
                k += 1
                raise ValueError("Invalid Input: Attacker {} has no more units".format(item))
            
            if defender['Current_Unit_Size'].values[0] <= 0:
                k += 1
                raise ValueError("Invalid Input: The Defender of the Attacker {} has no more units".format(item))
            
            if attacker['Current_HP'].values[0] <= 0:
                k += 1
                raise ValueError("Invalid Input: Attacker {} has no more HP".format(item))

            if defender['Current_HP'].values[0] <= 0:
                k += 1
                raise ValueError("Invalid Input: The Defender of the Attacker {} has no more HP".format(item))
            
            if attacker_range == 0 and given_range != 0:
                k += 1
                raise ValueError("Invalid input: Attacker {} should have a range of 0".format(item))

            # Simulation Precision
            if simulation_precision == 'max'or attacker['Current_Unit_Size'].values[0] < simulation_precision:
                Simulation_Precision.append(attacker['Current_Unit_Size'].values[0])
            else:
                Simulation_Precision.append(simulation_precision)
            
            attacker_unit_size = attacker['Current_Unit_Size'].values[0]
            max_unit_size = attacker['Max_Unit_Size'].values[0]
            APT = attacker['APT'].values[0]
            remaining_attacks = math.ceil(Simulation_Precision[index - k] * attacker_unit_size / max_unit_size)
            if given_range == 0:  # Melee attack
                attacker_damage = attacker['BD'].values[0]
                attacker_attack_type = 'AP'
            else:  # Ranged attack
                attacker_attack_type = 'MAP'
                if 0 < given_range <= round(0.2 * attacker_range / 1.5) *1.5:
                    attacker_attack = 1/2 * attacker_attack
                    attacker_damage = attacker['BMD'].values[0]
                elif round(0.2 * attacker_range / 1.5) *1.5 < given_range <= round(0.7 * attacker_range / 1.5) *1.5:
                    attacker_attack = 2 * attacker_attack
                    attacker_damage = 2 * attacker['BMD'].values[0]
                elif round(0.7 * attacker_range / 1.5) *1.5 < given_range <= attacker_range:
                    attacker_attack = attacker_attack
                    attacker_damage = 1/2 * attacker['BMD'].values[0]
                else:
                    raise ValueError("Invalid Input: Defender is Outside the range of the Attacker {}".format(item)) 
                

            chance_to_hit = 35 + attacker_attack - defender_defense

            attack_results = []
            attack_damage = []
            total_damage = [0] * APT
            
            for j in range(APT):
                for i in range(Simulation_Precision[index - k]):
                    if i <= remaining_attacks - 1:
                        roll = random.uniform(0.5, 1.0)
                        if defender['Armor'].values[0] * roll <= 100: 
                            defense_multiplier = 100 - defender['Armor'].values[0] * roll
                        else:
                            defense_multiplier = 0        
                        damage = attacker['AOE'].values[0] * (attacker_damage * defense_multiplier /100 + attacker[attacker_attack_type].values[0])
                        attack_result = 1 if random.randint(1, 100) <= chance_to_hit else 0
                        attack_results.append(attack_result)
                        attack_damage.append(math.ceil(damage) if attack_result == 1 else 0)
                        if i == remaining_attacks - 1: 
                            total_damage[j] = total_damage[j] + attack_damage[i+j]*(attacker_unit_size - (remaining_attacks-1)*math.floor(max_unit_size/Simulation_Precision[index - k]))
                        else:
                            total_damage[j] = total_damage[j] + attack_damage[i+j]*math.floor(attacker_unit_size/Simulation_Precision[index - k])
                    else:
                        attack_results.append(0)
                        attack_damage.append(0)
            
            total_damage = sum(total_damage)
            defender_hp = defender['Current_HP'].values[0]
            
            if attacker['Max_Unit_Size'].values[0] <= 10:
                PIS = 1

            units_lost = math.floor(PIS*total_damage / defender_hp) 
            
            total_damage_to_HP = (1-PIS)*total_damage + ((PIS)*total_damage) % defender_hp
            HP_lost = math.floor(total_damage_to_HP / defender['Current_Unit_Size'].values[0])

            dataframe.loc[[defender_type[index]], 'Current_Unit_Size'] -= units_lost
            dataframe.loc[[defender_type[index]], 'Current_HP'] -= HP_lost

            if dataframe.loc[[defender_type[index]], 'Current_HP'].values[0] <= 0 or dataframe.loc[[defender_type[index]], 'Current_HP'].values[0] <= 0:
                Unit_That_Died.append(defender_type[index])

            Chance_To_Hit.append(chance_to_hit)
            Attack_Results.append(attack_results)
            Attack_Damage.append(attack_damage)
            Units_Lost.append(units_lost)
            HP_Lost.append(HP_lost)
            Index.append(attacker_type[index])
            
            
        except ValueError as e:
            Non_Attacker.append(attacker_type[index])
            Non_Defender.append(defender_type[index])
            Non_Range.append(Given_Range[index])
            if str(e) == "Invalid Input: Attacker {} has no more units".format(item):
                print("Attacker Unit Error:", str(e))
            elif str(e) == "Invalid Input: The Defender of the Attacker {} has no more units".format(item):
                print("Defender Unit Error:", str(e))
            elif str(e) == "Invalid Input: Attacker {} has no more HP".format(item):
                print("Attacker Unit Error:", str(e))
            elif str(e) == "Invalid Input: The Defender of the Attacker {} has no more units".format(item):
                print("Defender Unit Error:", str(e))
            elif str(e) == "Invalid input: Attacker {} should have a range of 0".format(item):
                print("Range Unit Error:", str(e))
            elif str(e) == "Invalid Input: Defender is Outside the range of the Attacker {}".format(item):
                print("Range Unit Error:", str(e))
            else:
                print("Unknown ValueError:", str(e))
    
    

    if len(Non_Attacker) != 0:
        print("\nattacker_type =", Non_Attacker)
        print("defender_type =", Non_Defender)
        print("unit_range =", Non_Range)

    
    print("\nThe Attackers that Attacked are", Index)
    print("Chance to hit:", Chance_To_Hit)
    print("Simulation Precision:", Simulation_Precision)
    print("Attack results:")
    print('\n'.join([str(sublist) for sublist in Attack_Results]))
    print("Attack Damage:")
    print('\n'.join([str(sublist) for sublist in Attack_Damage]))
    print("Units Lost:", Units_Lost)
    print("HP lost", HP_Lost)
    print("Troops that Died", Unit_That_Died)

    return 


In [1409]:
attacker_type = [39]
defender_type = [8]
unit_range = [0] * 2


calculate_chance_to_hit(attacker_type, defender_type, data_frame, unit_range, 'max')



[410, 410, 458]
1278

The Attackers that Attacked are [39]
Chance to hit: [54]
Simulation Precision: [1]
Attack results:
[1, 1, 1]
Attack Damage:
[410, 410, 458]
Units Lost: [9]
HP lost [3]
Troops that Died []


In [1410]:
data_frame

,Unit_Type,Max_Unit_Size,Current_Unit_Size,Max_HP,Current_HP,Speed,Attack,Defense,BD,AP,BMD,MAP,AOE,Range,RT,Armor,W,BL,Size,APT
1,Fanteria_Leggera_1,32,32,62,62,3.0,33,30,29,5,0,0,1,0.0,0,60,Fire,0,1,1
2,Fanteria_Leggera_2,32,32,62,62,3.0,33,30,29,5,0,0,1,0.0,0,60,Fire,0,1,1
3,Fanteria_Leggera_3,32,32,62,62,3.0,33,30,29,5,0,0,1,0.0,0,60,Fire,0,1,1
4,Fanteria_Leggera_4,32,32,62,62,3.0,33,30,29,5,0,0,1,0.0,0,60,Fire,0,1,1
5,Fanteria_Leggera_5,32,32,62,62,3.0,33,30,29,5,0,0,1,0.0,0,60,Fire,0,1,1
6,Fanteria_Pesante_1,24,24,108,108,1.5,38,28,29,13,0,0,1,0.0,0,50,No,8,1,1
7,Fanteria_Pesante_2,24,24,108,108,1.5,38,28,29,13,0,0,1,0.0,0,50,No,8,1,1
8,Fanteria_Elite_1,16,7,135,132,1.5,40,31,14,36,0,0,1,0.0,0,110,No,0,1,1
9,Arcieri_1,24,24,55,55,3.0,16,14,19,5,16,3,1,12.0,0,15,Fire,0,1,1
10,Arcieri_2,24,24,55,55,3.0,16,14,19,5,16,3,1,12.0,0,15,Fire,0,1,1


In [1411]:
#data_frame['Max_HP'] = round(data_frame['Max_HP'] * 2 / 5)
#print(data_frame[['Unit_Type','Max_HP']])

In [1412]:
data_frame.loc[32,'Current_HP'] -= 0